In [1]:
import fitz
def pdf_extract(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text
india = pdf_extract('india/India_7.pdf') +" "+ pdf_extract('india/India_8.pdf')+" "+ pdf_extract('india/India_9.pdf')
malay = pdf_extract('malay/malay_7.pdf')
thai = pdf_extract('thailand/thai_7.pdf')

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: unknown cid font type

MuPDF error: syntax error: could not parse color space (240438 0 R)



In [2]:
textbooks = [india,malay,thai]

# Data Cleaning

In [3]:
import re
from langdetect import detect
from langdetect import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
def clean_text(post):
    if detect(post) != 'en':
        return ''
    # Make posts lowercase
    post = post.lower()

    # Remove punctuation
    post = re.sub(r'[^a-zA-Z\s]', ' ', post)
    
    # Remove words with repeated letters
    #post = re.sub(r'([a-zA-Z])\1+', r'\1', post)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(post)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

In [5]:
cleanTextbooks = []
for i in textbooks:
    cleanTextbooks.append(clean_text(i))

In [6]:
#These have spaces, we use these to select the male and female words
menWords = [" man ", " boy ", " male ", " brother ", " father ", " son ", " husband ", " king ", " prince ", " uncle ", " nephew ", " he ", " him ", " his ", " gentleman ", " sir ", " mr. ", " hero ", " lord ", " patriarch ", " men "]
womenWords = [" woman ", " girl ", " female ", " sister ", " mother ", " daughter ", " wife ", " queen ", " princess ", " aunt ", " niece ", " she ", " her ", " hers ", " lady ", " ma'am "," madam ", " mrs. ", " ms. ", " miss ", " heroine ", " dame ", " matriarch ", " women "]
#These have no spaces, so these can be used to throw away words during TFIDF
menWordsNoSpace = ["man", "boy", "male", "brother", "father", "son", "husband", "king", "prince", "uncle", "nephew", "he", "him", "his", "gentleman", "sir", "mr", "hero", "lord", "patriarch", "men"]
womenWordsNoSpace = ["woman", "girl", "female", "sister", "mother", "daughter", "wife", "queen", "princess", "aunt", "niece", "she", "her", "hers", "lady", "ma'am","madam", "mrs", "heroine", "dame", "matriarch", "women"]

In [7]:
menPara = [[] for _ in range(3)]
womenPara = [[] for _ in range(3)]
for i in range(0, 3):
    text = cleanTextbooks[i]
    paragraphs = [text[j:j+100] for j in range(0, len(text), 100)]
    for paragraph in paragraphs:
        for j in menWords:
            if j in paragraph:
                menPara[i].append(paragraph)
        for j in womenWords:
            if j in paragraph:
                womenPara[i].append(paragraph)

# Occurence

In [8]:
men_count = sum(len(sublist) for sublist in menPara)
print(men_count)
women_count = sum(len(sublist) for sublist in womenPara)
print(women_count)

430
245


# Firstness

In [9]:
femaleFirst = []
for sublist in womenPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1): 
                if words[i].lower() in womenWordsNoSpace and words[i + 1].lower() in menWordsNoSpace:
                    femaleFirst.append((words[i], words[i + 1]))
print(len(femaleFirst),femaleFirst)

5 [('daughter', 'son'), ('woman', 'man'), ('mother', 'father'), ('mother', 'brother'), ('women', 'men')]


In [10]:
maleFirst = []
for sublist in menPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1):  
                if words[i].lower() in menWordsNoSpace and words[i + 1].lower() in womenWordsNoSpace:
                    maleFirst.append((words[i], words[i + 1]))
print(len(maleFirst),maleFirst)

14 [('man', 'wife'), ('uncle', 'aunt'), ('uncle', 'aunt'), ('son', 'daughter'), ('son', 'daughter'), ('men', 'women'), ('father', 'mother'), ('father', 'mother'), ('father', 'mother'), ('brother', 'sister'), ('boy', 'girl'), ('father', 'mother'), ('male', 'female'), ('father', 'mother')]


In [11]:
sameLevel = {
    "siblings": {"brother", "sister"},
    "parent_child": {"father", "mother", "son", "daughter"},
    "spouses": {"husband", "wife"},
    "royalty": {"king", "queen", "prince", "princess"},
    "extended_family": {"uncle", "aunt", "nephew", "niece"},
    "personal_pronouns": {"he", "she", "him", "her", "his", "hers"},
    "titles": {"gentleman", "lady", "sir", "ma'am", "mr", "mrs", "madam"},
    "heroic_roles": {"hero", "heroine"},
    "social_roles": {"patriarch", "matriarch"},
    "general_plural": {"men", "women"},
}
flat = {word for group in sameLevel.values() for word in group}
def isSame(pair):
    return pair[0] in flat and pair[1] in flat
femaleFirst = [pair for pair in femaleFirst if isSame(pair)]

In [12]:
maleFirst = [pair for pair in maleFirst if isSame(pair)]
print(len(maleFirst))
print(len(femaleFirst))

11
4


# Sentiment analysis

In [13]:
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
fscores = analyzer.polarity_scores(fw)
mscores = analyzer.polarity_scores(fm)
print(fscores,mscores)

{'neg': 0.1, 'neu': 0.75, 'pos': 0.15, 'compound': 0.9997} {'neg': 0.089, 'neu': 0.743, 'pos': 0.169, 'compound': 1.0}


# TFIDF

In [14]:
from langdetect import detect
from langdetect import LangDetectException
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
#stemming the words
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_nested_list(nested_list):
    return [
        [
            ' '.join(stemmer.stem(word) for word in sentence.split())
            for sentence in sublist
        ]
        for sublist in nested_list
    ]
menParaOriginal = menPara
womenParaOriginal = womenPara
menPara = stem_nested_list(menPara)
womenPara = stem_nested_list(womenPara)
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))

In [16]:
tobeTFIDF = [" ".join(" ".join(row) for row in menPara)," ".join(" ".join(row) for row in womenPara)]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tobeTFIDF)
feature_names = vectorizer.get_feature_names_out()
dense_tfidf = tfidf_matrix.toarray()

In [17]:
#tfidf of female words
doc_index = 1
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
top_300_words_fs = sorted_word_scores[:300]
top_300_words_f = [word for word, score in sorted_word_scores[:300]]

In [18]:
#tfidf of male words
doc_index = 0  
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs_m = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores_m = sorted(word_score_pairs_m, key=lambda x: x[1], reverse=True)
top_300_words_ms = sorted_word_scores_m[:300]
top_300_words_m = [word for word, score in sorted_word_scores_m[:300]]

In [19]:
#remove common words and gendered keywords
common = list(set(top_300_words_f) & set(top_300_words_m))
top_300_words_f = [word for word in top_300_words_f if word not in common]
top_300_words_f = [word for word in top_300_words_f if word not in womenWordsNoSpace]
female_300 = " ".join(top_300_words_f)
top_300_words_m = [word for word in top_300_words_m if word not in common]
top_300_words_m = [word for word in top_300_words_m if word not in menWordsNoSpace]
male_300 = " ".join(top_300_words_m)

In [20]:
print(female_300)
print(male_300)

ead ou ladi miss left tenni exampl life reach santosh nd saint scale suicid big cri garden grandmoth heart note show stay tilli tsunami world kitchen love margi save sit war hyster iyer peter puan adriana alon began bless born cook dear eat everi fear femal finish five food gener hospit invit island learn maria op paper sat sunday though top train whether betti bijju brink chair cost howev manag marri quick regard sheela shout sinc siti sivasubramania smith wretch accept angri anyth babi becam caught consid countri differ draw er everest happen hilsa hold idea keep meal mileva mosquito name nice number paragraph send sent serv sharapova short speak spot start summit surpris tabl vii wait watch week afternoon appear arm author aw beach beehiv begin benara bit car chapter chop describ dialogu discuss en english es etc explain faint fath full geographi giant green half inherit laugh learnt leg
hermit made wound jodi wise shop bepin beard sound never enemi taken knew gopal last walk well b

In [21]:
from collections import Counter
import re
mwords = male_300.split()
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)

In [22]:
fwords = female_300.split()
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)

In [23]:
from collections import Counter
import re

# Example word lists (assuming male_300 and female_300 are defined as strings)
mwords = male_300.split()  # Male words
fwords = female_300.split()  # Female words

# Filtered words for male
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)
print("Top 40 Male Words:", topMwords)

# Filtered words for female
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)
print("Top 40 Female Words:", topFwords)


Top 40 Male Words: [('hermit', 1), ('made', 1), ('wound', 1), ('jodi', 1), ('wise', 1), ('shop', 1), ('bepin', 1), ('beard', 1), ('sound', 1), ('never', 1), ('enemi', 1), ('taken', 1), ('knew', 1), ('gopal', 1), ('last', 1), ('walk', 1), ('well', 1), ('bed', 1), ('mean', 1), ('penni', 1), ('import', 1), ('ing', 1), ('long', 1), ('money', 1), ('other', 1), ('pass', 1), ('round', 1), ('turn', 1), ('ye', 1), ('forgiv', 1), ('fruit', 1), ('grey', 1), ('happi', 1), ('pleas', 1), ('land', 1), ('mani', 1), ('put', 1), ('sure', 1), ('tri', 1), ('along', 1)]
Top 40 Female Words: [('ead', 1), ('ou', 1), ('ladi', 1), ('miss', 1), ('left', 1), ('tenni', 1), ('exampl', 1), ('life', 1), ('reach', 1), ('santosh', 1), ('nd', 1), ('saint', 1), ('scale', 1), ('suicid', 1), ('big', 1), ('cri', 1), ('garden', 1), ('grandmoth', 1), ('heart', 1), ('note', 1), ('show', 1), ('stay', 1), ('tilli', 1), ('tsunami', 1), ('world', 1), ('kitchen', 1), ('love', 1), ('margi', 1), ('save', 1), ('sit', 1), ('war', 1), 

# random forest

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)


proba = clf.predict_proba(X)[:, 1]  # Probability of being female
female_contributions = np.dot(X.T.toarray(), proba)
female_importance = female_contributions / np.sum(X.toarray(), axis=0)
female_importance_df = pd.DataFrame({
    "Word": feature_names,
    "FemaleImportance": female_importance
}).sort_values(by="FemaleImportance", ascending=False)

print("Top words most related to Female:")
print(female_importance_df.head(10))

Top words most related to Female:
        Word  FemaleImportance
704      ead          1.000000
809   exampl          1.000000
1748      ou          1.000000
1621      nd          1.000000
1192  hyster          0.994192
1832   peter          0.994000
2137   saint          0.993601
2267    siti          0.988652
1286    iyer          0.988370
2423  suicid          0.987958


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)

proba = clf.predict_proba(X)[:, 0]  # Probability of being male (class 0)


male_contributions = np.dot(X.T.toarray(), proba)
male_importance = male_contributions / np.sum(X.toarray(), axis=0)
male_importance_df = pd.DataFrame({
    "Word": feature_names,
    "MaleImportance": male_importance
}).sort_values(by="MaleImportance", ascending=False)

print("Top words most related to Male:")
print(male_importance_df.head(10))

Top words most related to Male:
            Word  MaleImportance
2331       sound             1.0
1586    mountain             1.0
2609          un             1.0
757        enemi             1.0
1536     mention             1.0
2447     syllabl             1.0
694          dug             1.0
1083       happi             1.0
2791       wound             1.0
1989  rameswaram             1.0


In [30]:
fm = ', '.join(map(str, [item for sublist in menParaOriginal for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenParaOriginal for item in sublist]))
ps = PorterStemmer()
words = word_tokenize(fw)
matching = [word for word in words if ps.stem(word) in {"cass", "aborigin","sungen"}]
matching

[]